In [2]:
# Modell 3: Tren på værtype fra CSV
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
import joblib

# Les og forbered data
df = pd.read_csv("../data/processed/Ulykker_med_værforhold.csv", encoding="ISO-8859-1", sep=";")
df.columns = df.columns.str.strip()
df["Kommune"] = df["Kommune"].str.split(" - ").str[-1].str.strip().str.lower()

# Smelt værtyper til én kolonne
melted = df.melt(
    id_vars=["Kommune", "Måned", "År"],
    var_name="Værtype",
    value_name="AntallUlykker"
).dropna()

melted["Værtype"] = melted["Værtype"].str.lower()

# Lag risikonivå
melted["Risiko"] = pd.cut(melted["AntallUlykker"],
                          bins=[-1, 0, 2, float("inf")],
                          labels=["lav", "middels", "høy"])

# Modelltrening
X = melted[["Kommune", "Måned", "År", "Værtype"]]
y = melted["Risiko"]

pipeline = Pipeline([
    ("prep", ColumnTransformer([
        ("cat", OneHotEncoder(handle_unknown="ignore"), ["Kommune", "Måned", "Værtype"]),
        ("num", "passthrough", ["År"])
    ])),
    ("clf", RandomForestClassifier(n_estimators=50, random_state=42))
])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
pipeline.fit(X_train, y_train)

# Lagre modellen
joblib.dump(pipeline, "modell_vaerforhold_pipeline.pkl")


['modell_vaerforhold_pipeline.pkl']

In [4]:
def kategoriser_vaer(mean_temp, sum_precip, mean_wind):
    if sum_precip == 0 and mean_wind < 6:
        return "god sikt, opphold"
    elif 0 < sum_precip <= 5 and mean_wind < 6:
        return "god sikt, nedbør"
    elif sum_precip > 5:
        return "dårlig sikt, nedbør"
    elif sum_precip == 0 and mean_temp < 3 and mean_wind < 3:
        return "dårlig sikt, tåke eller dis"
    elif mean_wind >= 6:
        return "dårlig sikt, for øvrig"
    else:
        return "god sikt, opphold"

def prediker_fra_met(kommune, måned, år, mean_temp, sum_precip, mean_wind):
    kommune_renset = kommune.strip().lower()
    værtype = kategoriser_vaer(mean_temp, sum_precip, mean_wind)
    
    input_df = pd.DataFrame([[kommune_renset, måned, år, værtype]],
                            columns=["Kommune", "Måned", "År", "Værtype"])
    return pipeline.predict(input_df)[0]

In [5]:
risiko = prediker_fra_met(
    kommune="Oslo",
    måned="Mars",
    år=2024,
    mean_temp=2.1,
    sum_precip=0.0,
    mean_wind=1.5
)

print("Predikert risikonivå:", risiko)

Predikert risikonivå: høy
